In [ ]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d2b9ad9116e22a42a1d9e40d289ef22dcb715fde25ea8218d99fa8fd870ece40
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import nltk
import evaluate
import numpy as np
from datasets import Dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Carregar o tokenizer, modelo e data collator
MODEL_NAME = "google/flan-t5-large"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
import json

uploaded = files.upload()

with open('/content/dataset.json', 'r') as f:
    dataset = json.load(f)


Saving dataset.json to dataset.json


In [ ]:
from collections import defaultdict

cantos_por_tempo = defaultdict(lambda: defaultdict(list))
for canto in dataset:
    cantos_por_tempo[canto['tempo']][canto['parte']].append(canto)

# Separar um canto de cada categoria para validação
validacao = []
for tempo, partes in cantos_por_tempo.items():
    for parte, cantos in partes.items():
        if cantos:
            validacao.append(cantos.pop(0))

# Organizar o restante como dados de treino
treino = [canto for partes in cantos_por_tempo.values() for cantos in partes.values() for canto in cantos]

# Converter para Dataset da Hugging Face
train_dataset = Dataset.from_dict({"lyric": [c["lyric"] for c in treino], "tempo": [c["tempo"] for c in treino]})
val_dataset = Dataset.from_dict({"lyric": [c["lyric"] for c in validacao], "tempo": [c["tempo"] for c in validacao]})


In [ ]:
# Prefixo para a tarefa de classificação
prefix = "Classifique o seguinte canto nos tempos: natal, pascoa, quaresma, comum e advento: ";

# Função de preprocessamento
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["lyric"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Tokenizar os labels (os tempos litúrgicos)
    labels = tokenizer(text_target=examples["tempo"], max_length=32, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Aplicar o preprocessamento
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result


In [ ]:
# Parâmetros globais
L_RATE = 2e-5
BATCH_SIZE = 2
PER_DEVICE_EVAL_BATCH = 6
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 50  # Tente com 20 ou 10

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    lr_scheduler_type="cosine",
    predict_with_generate=True,
    push_to_hub=False
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import torch
torch.cuda.empty_cache()


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Iniciar o treinamento
trainer.train()

# Avaliar o modelo treinado
eval_results = trainer.evaluate()
print(eval_results)

# Salvar o modelo treinado
trainer.save_model("./modelo_finetuned")


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,1.278704,0.550000,0.000000,0.550000,0.550000
2,No log,1.737255,0.500000,0.000000,0.500000,0.500000
3,No log,1.309451,0.550000,0.000000,0.550000,0.550000
4,No log,2.477167,0.500000,0.000000,0.500000,0.500000
5,No log,2.113635,0.500000,0.000000,0.500000,0.500000
6,No log,2.132020,0.550000,0.000000,0.550000,0.550000
7,No log,2.322129,0.500000,0.000000,0.500000,0.500000
8,No log,1.773630,0.500000,0.000000,0.500000,0.500000
9,No log,1.872480,0.600000,0.000000,0.600000,0.600000
10,No log,2.349914,0.550000,0.000000,0.550000,0.550000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control

{'eval_loss': 4.005788803100586, 'eval_rouge1': 0.5, 'eval_rouge2': 0.0, 'eval_rougeL': 0.5, 'eval_rougeLsum': 0.5, 'eval_runtime': 3.6433, 'eval_samples_per_second': 5.49, 'eval_steps_per_second': 1.098, 'epoch': 50.0}


In [ ]:

from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Carregar o modelo e o tokenizer
model_path = "./modelo_finetuned"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Colocar o modelo em modo de avaliação
model.eval()

# Lista para armazenar as previsões e rótulos reais
predictions = []
true_labels = []

# Definir um prefixo para a tarefa de classificação
prefix = "Classifique o seguinte canto nos tempos: natal, pascoa, quaresma, comum e advento: ";

# Realizar previsões
with torch.no_grad():
    for example in val_dataset:
        input_text = prefix + example["lyric"]
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids

        outputs = model.generate(input_ids)
        predicted_class = tokenizer.decode(outputs[0], skip_special_tokens=True)

        predictions.append(predicted_class)
        true_labels.append(example["tempo"])

# Agora você pode avaliar os resultados usando as métricas que você escolher
from sklearn.metrics import classification_report

print(classification_report(true_labels, predictions))



Token indices sequence length is longer than the specified maximum sequence length for this model (598 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


              precision    recall  f1-score   support

     advento       0.67      1.00      0.80         4
       comum       0.33      0.25      0.29         4
       natal       1.00      0.75      0.86         4
      pascoa       0.50      0.50      0.50         4
    quaresma       0.50      0.50      0.50         4

    accuracy                           0.60        20
   macro avg       0.60      0.60      0.59        20
weighted avg       0.60      0.60      0.59        20



In [ ]:
!zip -r modelo_finetuned.zip ./modelo_finetuned
from google.colab import files
files.download('modelo_finetuned.zip')

  adding: modelo_finetuned/ (stored 0%)
  adding: modelo_finetuned/model.safetensors (deflated 7%)
  adding: modelo_finetuned/generation_config.json (deflated 29%)
  adding: modelo_finetuned/training_args.bin (deflated 51%)
  adding: modelo_finetuned/added_tokens.json (deflated 83%)
  adding: modelo_finetuned/spiece.model (deflated 48%)
  adding: modelo_finetuned/special_tokens_map.json (deflated 85%)
  adding: modelo_finetuned/tokenizer_config.json (deflated 94%)
  adding: modelo_finetuned/config.json (deflated 48%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>